# Projeto 2 - Classificador Automático de Sentimento 

### Renato Tajima e Thiago Verardo



Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

# Analisando os tweets do mouse G403


# Importando os tweets a serem analisados


Nesse item nós importamos uma tabela de tweets e os classificamos, manualmente, um a um se eles são relevantes ou não para o desenvolvimento do produto, ou seja, se eles constituem uma crítica positiva ou negativa, com o objetivo de eles servirem como uma forma de modelo para o nosso código no final.



In [10]:
import pandas as pd
import numpy as np
from emoji import UNICODE_EMOJI

In [11]:
Treinamento = pd.read_excel('tweets_treinamento.xlsx')
Teste = pd.read_excel('Tweets_teste.xlsx')


Treinamento.head()

,Treinamento,Class
0,"best gaming mouse-logitech g903,g403,g703 and ...",1
1,rt @tsmviss: new giveaway!\ngiving away 3 sets...,0
2,rt @gdw444: 2018-2019 mpp students @policy_sch...,0
3,rt @imnategibson: g403 paracorded. https://t.c...,0
4,rt @tsmviss: new giveaway!\ngiving away 3 sets...,0


# Limpando os tweets

Após importamos a tabelas, temos que retirar alguns caracteres que podem atrapalhar o classificador, como pontos, espaços, virgulas, tab, três pontos, enters, #, http, rt e @.

In [12]:
import nltk
import string
pontuacao = string.punctuation

#Primeira Limpada

#lista de itens indesejáveis (pontos, espaços, virgulas, tab, três pontos e enters)
pont = ["," ,"\n", "\t", ".", "…"]

tweet = Treinamento["Treinamento"]

tweets_limpos = []

for e in tweet:
    a = ""
    for i in e:
        if i in UNICODE_EMOJI:
            a = a+" "+i+" "
        elif i in pont:
            a += " "
        elif i not in pontuacao:
            a+= i
    tweets_limpos.append(a)
    
g403_limpo = pd.DataFrame()
g403_limpo["tweets"] = tweets_limpos
g403_limpo["Relevância"] = Treinamento["Class"]

g403_limpo.head()

,tweets,Relevância
0,best gaming mouselogitech g903 g403 g703 and g...,1
1,rt tsmviss new giveaway giving away 3 sets of ...,0
2,rt gdw444 20182019 mpp students policyschool u...,0
3,rt imnategibson g403 paracorded httpst coq0op...,0
4,rt tsmviss new giveaway giving away 3 sets of ...,0


In [13]:
#Segunda Limpada
#Retirando os #, http, rt e @.

tweet_limpo = []
y = " "
for tweet in g403_limpo["tweets"]:
    limpo = []
    split = tweet.split(" ")
    for e in split:
        if e in UNICODE_EMOJI:
            limpo.append(e)
        elif len(e)>2 and e[0] != "@" and e[0] != "#" and e[0] != "rt" and e[:4] != "http":
            limpo.append(e)
    tweet_limpo.append(y.join(limpo))
    
g403_pronto = pd.DataFrame()
g403_pronto["tweets"] = tweet_limpo
g403_pronto["Relevância"] = Treinamento["Class"]

g403_pronto.head()

,tweets,Relevância
0,best gaming mouselogitech g903 g403 g703 and g...,1
1,tsmviss new giveaway giving away sets peripher...,0
2,gdw444 20182019 mpp students policyschool ucal...,0
3,imnategibson g403 paracorded coq0opm8kh4v,0
4,tsmviss new giveaway giving away sets peripher...,0


# Lógica de classificação


Para desesnvolver o nosso classificador nós colocamos as palavras dos tweets em uma lista com o método split e depois a comparamos com a nossa tabela tweets, para analisarmos quais as palavras que aparecem mais nos tweets relevantes e irrelevantes.

In [14]:
palavras = []

for e in g403_pronto["tweets"]:
    split = e.split(" ")
    for i in split:
        if i not in palavras:
            palavras.append(i)
palavras_relevantes = 0
palavras_irrelevantes = 0
for e in range(len(g403_pronto["tweets"])):
    split = g403_pronto["tweets"][e].split(" ")
    if g403_pronto["Relevância"][e] == 1:
        for i in split:
            palavras_relevantes += 1
    else:
        for i in split:
            palavras_irrelevantes += 1

# Calculando a probabilidade

Com as palavras separadas e analisadas, podemos calcular P( palavra | relevante ) e P( palavra | não relevante ) e P( relevante ) e P( não relevante ):

In [15]:
relevante = {}
irrelevante = {}

for e in palavras:
    relevante[e] = 1
    irrelevante[e] = 1

for i in range(len(g403_pronto["tweets"])):
    split = g403_pronto["tweets"][i].split(' ')
    if g403_pronto['Relevância'][i] == 1:
        for a in split:
            relevante[a] += 1
    else:
        for b in split:
            irrelevante[b] += 1

In [16]:
prob_rel = {}
prob_irrel = {}

for e in palavras:
    prob_rel[e] = relevante[e] / (palavras_relevantes + len(palavras))
    prob_irrel[e] = irrelevante[e] / (palavras_irrelevantes + len(palavras))
    
s_rel = 0
s_irrel = 0
for i in g403_pronto["Relevância"]:
    if i == 0:
        s_irrel +=1
    else:
        s_rel +=1
        
prob_relevante = s_rel / len(g403_pronto["Relevância"])
prob_irrelevante = s_irrel / len(g403_pronto["Relevância"])

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

Para isso devemos refazer o processo com um dataframe novo, nesse caso foi o "Teste" (previamente criado). 


In [17]:
Teste.head()

,Teste,Class
0,looking forward to having @tomclarkgpa in #yyc...,0
1,rt @tsmviss: new giveaway!\ngiving away 3 sets...,0
2,rt @tsmviss: new giveaway!\ngiving away 3 sets...,0
3,rt @tsmviss: new giveaway!\ngiving away 3 sets...,0
4,rt @tsmviss: new giveaway!\ngiving away 3 sets...,0


Limpá-lo, como o outro.

In [18]:
import nltk
import string
pontuacao = string.punctuation

pont = ["," ,"\n", "\t", "."]

tweet = Teste["Teste"]

tweets_limpos = []

for e in tweet:
    a = ""
    for i in e:
        if i in UNICODE_EMOJI:
            a = a+" "+i+" "
        elif i in pont:
            a+= " "
        elif i not in pontuacao:
            a+= i
    tweets_limpos.append(a)
    
teste_limpo = pd.DataFrame()
teste_limpo["tweets"] = tweets_limpos
teste_limpo["Relevância"] = Teste["Class"]

tweet_limpo = []
y = " "
for tweet in teste_limpo["tweets"]:
    limpo = []
    split = tweet.split(" ")
    for e in split:
        if e in UNICODE_EMOJI:
            limpo.append(e)
        elif len(e)>2 and e[0] != "@" and e[0] != "#" and e[0] != "rt" and e[:4] != "http":
            limpo.append(e)
    tweet_limpo.append(y.join(limpo))
    
teste_pronto = pd.DataFrame()
teste_pronto["tweets"] = tweet_limpo
teste_pronto["Class"] = Teste["Class"]

teste_pronto.head()

,tweets,Class
0,looking forward having tomclarkgpa yyc lead th...,0
1,tsmviss new giveaway giving away sets peripher...,0
2,tsmviss new giveaway giving away sets peripher...,0
3,tsmviss new giveaway giving away sets peripher...,0
4,tsmviss new giveaway giving away sets peripher...,0


Com o dataframe limpo, podemos fazer a previsão se um tweet é relevante, ou não

In [19]:
previsao = []

for e in teste_pronto["tweets"]:
    prob_relev = 1
    prob_irrelev = 1
    palavras = e.split(" ")
    
    for i in palavras:
        if i in prob_rel:
            prob_relev *= prob_rel[i]
        else:
            prob_relev *= (1/(s_rel + len(palavras)))
        
    for i in palavras:
        if i in prob_irrel:
            prob_irrelev *= prob_irrel[i]
        else:
            prob_irrelev *= (1/(s_irrel + len(palavras)))
    relevantes = prob_relev * prob_relevante
    irrelevantes = prob_irrelev * prob_irrelevante
    
    if relevantes > irrelevantes:
        previsao.append(1)
    else:
        previsao.append(0)
        
teste_pronto["Previsão"] = previsao

teste_pronto.head()

,tweets,Class,Previsão
0,looking forward having tomclarkgpa yyc lead th...,0,1
1,tsmviss new giveaway giving away sets peripher...,0,0
2,tsmviss new giveaway giving away sets peripher...,0,0
3,tsmviss new giveaway giving away sets peripher...,0,0
4,tsmviss new giveaway giving away sets peripher...,0,0


Com isso feito, podemos calcular a Porcentagem de positivos falsos, Porcentagem de positivos verdadeiros, Porcentagem de negativos verdadeiros e Porcentagem de negativos falsos

In [20]:
crelevantes = 0
cirrelevantes = 0

for i in range(len(teste_pronto['Class'])):
    if teste_pronto['Class'][i] == 1:
        crelevantes += 1
    else:
        cirrelevantes += 1

prcr = 0
pircr = 0
prcir = 0
pircir = 0

for i in range(len(teste_pronto['Previsão'])):
    if teste_pronto['Previsão'][i] == 1 and teste_pronto['Class'][i] == 1:
        prcr += 1
    elif teste_pronto['Previsão'][i] == 0 and teste_pronto['Class'][i] == 1:
        pircr += 1
    elif teste_pronto['Previsão'][i] == 1 and teste_pronto['Class'][i] == 0:
        prcir += 1
    else:
        pircir += 1
        
P_pos_falso = prcir / cirrelevantes
P_pos_verdadeiro = prcr / crelevantes
P_neg_verdadeiro = pircir / cirrelevantes
P_neg_falso = pircr / crelevantes

Porcentagem de positivos falsos

In [21]:
P_pos_falso * 100
print(P_pos_falso)

0.12432432432432433


Porcentagem de positivos verdadeiros

In [22]:
P_pos_verdadeiro * 100
print(P_pos_verdadeiro)

1.0


Porcentagem de negativos verdadeiros

In [23]:
P_neg_verdadeiro * 100
print(P_neg_verdadeiro)

0.8756756756756757


Porcentagem de negativos falsos

In [24]:
P_neg_falso * 100
print(P_neg_falso)

0.0


E, por fim, calcular a porcentagem de acertos do classificador de tweets.

In [25]:
acertos = 0

for i in range(len(teste_pronto['Previsão'])):
    if teste_pronto['Previsão'][i] == teste_pronto['Class'][i]:
        acertos += 1
        
Precisao = acertos / len(teste_pronto)

print('O classificador acerta a relevância de um tweet em {0:.2f}% dos casos.'.format(Precisao * 100))

O classificador acerta a relevância de um tweet em 88.50% dos casos.


# ___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


# Conclusão


Nesse projeto nós buscamos criar um classificador de palavras, confiável, para se analisar algum produto específico que está sendo discutido no twitter, no nosso caso foi o mouse g403.

Para começar a fazer esse projeto nós importamos uma tabela de tweets que falava sobre o produto desejado. Após importá-la, conseguimos desenvolver uma lógica para conseguir classificar cada tweet dentro dela entre relevante ou irrelevante, ou seja, se o tweet faz uma crítica ao produto ou não.

Ao desenvolver o nosso classificador, nós consideramos todas as palavras relevantes dos tweets e os emojis presentes nele, para que assim nós possamos classificar as palavras em positivas verdadeiras (uma palavra que foi identificada como relevante e realmente é), positivas falsas (uma palavra que foi identificada como relevante e na verdade não é), negativo verdadeiro (uma palavra que foi identificada como irrelevante e realmente é) e negativo falso (uma palavra que foi identificada como irrelevante e é relevante). 

Após as suas classificações nós obtivemos que o nosso modelo possui um alto percentual de acertos entre os item relevantes, sendo ele de 88.50%, e o mais incrível ainda é que o nosso modelo consegue prever 100% dos tweets positivos e verdadeiros. Mostrando que o nosso modelo pode vir a ser muito útil para a avaliação de algum produto.



# Conclusão qualitativa sobre as medidas obtidas.

O nosso modelo de classificador, após analisar todos os tweets, conseguiu apresenta uma incrível precisão de 88,5% de detectção de tweets verdadeiros e relevantes, sendo que ele detecta 100% dos tweets positivos verdadeiros e, logicamente, 0% dos tweets verdadeiros negativos. Mostrando que nosso modelo é muito eficaz dentro de suas limitações

# Mensagens com dupla negação e sarcasmo

As mensagens com dupla negação e sacarsmos são complexas para se lidar com um classificador didático, uma vez que envolvem sentimentos e interpretações humanas. O nosso classificador não é capaz de detectar esses sentimentos, uma vez que ele só classifica as palavras e faz suas possibilidades, e não verifica o signifado por trás delas. A análise é feita somente com a palavra, não com o conjunto ou combinação delas.

# Plano de Expansão

Como dito anteriormente, nosso classificador apresentou um percentual muito elevado na detecção de tweets realmentes relevantes e o mais incrível ainda é que ele consegue prever 100% dos tweets classificados como positivos verdadeiros, mostrando que ele pode ser confiável e útil para qualquer empresa que venha a usá-lo no futuro.

Porém, como todo modelo, ele consegue ficar cada vez melhor e com ajuda de investimentos externos isso pode ser possível, uma vez que o dinheiro for aplicado, será possível criar um banco de dados maior, deixando ele muito mais preciso e uma melhoraria no filtro das palavras, com o objetivo de classificar o maior número de palavras que são classificadas como relevantes ou como irrelevantes, fazendo com que o classificador fique mais apurado. Além de uma aumentada no número de palavras "irrelevantes", que não agregam à relevância da resposta, para se ter um dataframe mais limpo e capaz de relacionar melhor as palavras.

Vê se que o potencial desse classificador é muito grande para empresas que recebem "feedback" e comentários de muitas pessoas todos os dias, pois o tempo de identificar comentários relevantes seria poupado, além de dar a opção delas darem o "feedback" ao consumidor/cliente mais rapidamente. 